<a href="https://colab.research.google.com/github/SubramDas/Homomorphic-Encryption-Secure-KNN/blob/main/KnnComp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install phe
import numpy as np
import time

total_encription_time = 0

time: 5.53 s (started: 2023-12-07 14:06:28 +00:00)


In [ ]:
# columns = 10
# rows = 10

# D = np.random.uniform(low=1, high=100, size=(rows, columns))
# D = np.round(D, 1)

# query = np.random.uniform(low=1, high=100, size=columns)
# query = np.round(query, 1)

# print(np.shape(D))



(10, 10)
time: 1.14 ms (started: 2023-12-07 14:06:33 +00:00)


**KEY** **GENERATION**

In [ ]:
#Key Generation

def generate_key(d, c, epsilon):

  #computing eta
  eta = d + c + epsilon + 1

  # Generate an base secret matrix Mbase of eta, eta
  Mbase = np.random.uniform(low=1, high=100, size=(eta, eta))

  Mbase = np.round(Mbase, 0)

  # Generate a (d + 1) dimensional long term secret vector S
  S = np.random.uniform(low=1, high=100, size=(d + 1))

  S = np.round(S, 0)
  # Generate a c-dimensional fixed vector w
  w = np.random.uniform(low=1, high=100, size=c)

  w = np.round(w, 0)
  # Generate a permutation function π of η numbers
  # pi = np.random.permutation(eta)

  #Secret key of the encryption scheme

  key = {
      'S': S,
      'Mbase': Mbase,
      'w': w
  }

  return key


time: 1.16 ms (started: 2023-12-07 14:17:35 +00:00)


**PARAMETER** **SELECTION**

In [ ]:
# Taking parameters

# d = len(D[0])
d = 3
c = 5
epsilon = 5

key = generate_key(d, c, epsilon)

time: 1.5 ms (started: 2023-12-07 14:17:45 +00:00)


**DATA** **ENCRYPTION**

In [ ]:

# Data Encryption
# D = the private database of DO consisting of m points
# D = {p1, p2, ... pm} where p1 = {pi1, pi2,... pid}
# ||pi|| = euclidean norm
# tic = time.time()

p1 = [10, 2.7, 5.3]
p2 = [5.3, 8.6, 2.5]
p3 = [1.3, 5.6, 8.5]

# # database D of n vectors with d dimensions ( n=1, d=2)
# # (Data Owner DO )
D = [p1, p2, p3]

# D = np.random.uniform(low = 1, high = 10, size=(2,2))

epsilon_dim = np.random.uniform(low = 1, high = 100, size=epsilon)
epsilon_dim = np.round(epsilon_dim, 0)

# ephemeral_secret_vector of length epsilon
z = epsilon_dim

def DataEnc(D, key):

  D_dash = []

  Mbase = key['Mbase']
  w = key['w']
  S = key['S']


  S_size = len(S) - 1

  # Collect S from 1 to d
  for i in range(len(D)):
    temp_pi_dash = []
    l2_norm_sq = 0

    # S till d
    for j in range(S_size):
      temp_pi_dash.append(S[j] - 2*D[i][j])
      l2_norm_sq += D[i][j]*D[i][j]

    temp_pi_dash.append(S[-1] + l2_norm_sq)

    for j in range(len(w)):
      temp_pi_dash.append(w[j])


    for k in range(epsilon):
      temp_pi_dash.append(z[k])

    # print(temp_pi_dash)
    M_inv = np.linalg.inv(Mbase)
    pi_dash = np.dot(temp_pi_dash, M_inv)

    D_dash.append(pi_dash)

  return D_dash

DataEnc = DataEnc(D, key)

# print(DataEnc)

# toc = time.time()
# total_encription_time += toc - tic

time: 3.22 ms (started: 2023-12-07 14:17:48 +00:00)


In [ ]:
# tic = time.time()

def euclidean_norm(vector):
    return np.linalg.norm(vector)

norms = [euclidean_norm(vector) for vector in D]

# print(D, norms)
max_norm = max(norms)

# toc = time.time()
# total_encription_time += toc - tic
# print(total_encription_time)

time: 933 µs (started: 2023-12-07 14:17:54 +00:00)


**QUERY ENCRYPTION (STEP 1)**

In [ ]:
tic = time.time()
# Query Encription
# Step1 performed by QU

query = [2.5, 3.1, 1.6]

# diagonal matrix Nd×d of real numbers
temp_values = np.random.uniform(low=1, high=100, size=d)
temp_values = np.round(temp_values, 0)

N = np.diag(temp_values)

# β1 is an encryption constant and is a real number
beta1 = np.round(100*np.random.rand())


print(beta1)

encrypted_query = beta1 * np.dot(query, N)

# print(encrypted_query)

toc = time.time()
total_encription_time += toc - tic
print(total_encription_time)


7.0
0.010700702667236328
time: 1.39 ms (started: 2023-12-07 14:18:19 +00:00)


**QUERY ENCRYPTION (STEP 2)**

In [ ]:
tic = time.time()
# Step2 query encryption is performed by the DO

qmax = max(encrypted_query)
# print(qmax)

eta = d + c + epsilon + 1

# temporary secret matrix Mt (η×η)
Mt = np.random.uniform(low = qmax+1, high=qmax+100, size=(eta, eta))
Mt = np.round(Mt, 0)

Mt_diag = np.random.uniform(low = max_norm+1, high = max_norm+100, size=eta)
Mt_diag = np.round(Mt_diag, 0)

for i in range(eta):
  Mt[i][i] = Mt_diag[i]

Msec = np.dot(Mt, key['Mbase'])

toc = time.time()
total_encription_time += toc - tic
print(total_encription_time)

0.011528491973876953
time: 3.4 ms (started: 2023-12-07 14:18:24 +00:00)


In [ ]:
tic = time.time()

q_dash = []

for i in range(len(encrypted_query)):
  q_dash.append(encrypted_query[i])

q_dash.append(1)

x = np.random.uniform(low = 1, high = 100, size=c)
x = np.round(x, 0)

for i in range(c):
  q_dash.append(x[i])

for i in range(epsilon):
  q_dash.append(0)

q_nn = np.diag(q_dash)

# print(q_nn)

E_matrix = np.random.uniform(low = qmax+1, high=qmax+100, size=(eta, eta))
E_matrix = np.round(E_matrix, 0)

beta2 = np.round(100*np.random.rand())

q_hat = (beta2 * (np.dot(Msec, q_nn) + E_matrix))

toc = time.time()
total_encription_time += toc - tic
print(total_encription_time)

0.01370549201965332
time: 8.85 ms (started: 2023-12-07 14:18:55 +00:00)


In [ ]:
# tic = time.time()
# CSP work
pi_doubledash = []
Mt_inv = np.linalg.inv(Mt)
for i in range(len(DataEnc)):
  pi_doubledash.append(np.dot(DataEnc[i], Mt_inv))

# toc = time.time()
# total_encription_time += toc - tic
# print(total_encription_time)

time: 5.57 ms (started: 2023-12-07 14:19:03 +00:00)


**QUERY ENCRYPTION (STEP 3)**

In [ ]:
tic = time.time()
# Step3

N_dash_temp_values = []

j=0
while j<d:
  N_dash_temp_values.append(temp_values[j]);
  j+=1

for i in range(eta-d):
  N_dash_temp_values.append(1)

N_dash = np.diag(N_dash_temp_values)

N_dash_inv = np.linalg.inv(N_dash)

q_tilda_enc = np.dot(q_hat, N_dash_inv)

# print(q_tilda_enc)

q_doubledash = []

for i in range(len(query)):
  q_doubledash.append(beta1*query[i])

q_doubledash.append(1)

for i in range(len(x)):
  q_doubledash.append(x[i])

for i in range(epsilon):
  q_doubledash.append(0)

q_doubledash = np.diag(q_doubledash)
q_dd = []

for i in range((eta)):
  q_dd.append(q_doubledash[i][i])

# temp = beta2 * (np.dot(Msec, q_doubledash) + np.dot(E_matrix, N_dash_inv))

# # print(temp == q_tilda_enc)
# q_tilda_enc = temp

err = []
dot_product = np.dot(E_matrix, N_dash_inv)
sum=0
for i in range(len(dot_product)):
  for j in range(len(dot_product[0])):
    sum = sum + dot_product[i][j]
  err.append(sum)

q_vec = []

for j in range(eta):
  sum_q_tilda_enc = 0
  for i in range(eta):
    sum_q_tilda_enc += q_tilda_enc[j][i]

  q_vec.append(sum_q_tilda_enc)

# print(q_vec)
q_vec = beta2 * (np.dot(Msec, q_dd) + err)

toc = time.time()
total_encription_time += toc - tic
print(total_encription_time)

0.01589345932006836
time: 8.06 ms (started: 2023-12-07 14:21:49 +00:00)


**DECRYPTION ALGORITHM**

In [ ]:
# Decryption
p_tilda_i = []
for i in range(len(DataEnc)):
  p_tilda_i.append(np.dot(DataEnc[i], key['Mbase']))


data_tuples = []

S = key['S']

for i in range(len(p_tilda_i)):
  row = []
  for j in range(d):
    value = np.round((S[j] - p_tilda_i[i][j])/2, 2)
    row.append(value)

  data_tuples.append(row)
print(data_tuples)


[[10.0, 2.7, 5.3], [5.3, 8.6, 2.5], [1.3, 5.6, 8.5]]
time: 1.99 ms (started: 2023-12-07 14:21:54 +00:00)


**KNN COMPUTATION**

In [ ]:
# KNN computation
# %%timeit

# knn = []
# print(np.shape(pi_doubledash))
# for i in range(len(p_tilda_i)):

#   erri = np.dot(pi_doubledash[i], err)
#   temp = beta2 * (np.dot(p_tilda_i, q_dd) + erri)
#   knn.append(beta2 * (np.dot(p_tilda_i, q_dd) + erri))



# tic = time.time()
knn = []
for i in range(len(pi_doubledash)):
  knn.append([(np.dot(pi_doubledash[i], q_vec)), i])

print(knn)
sorted_array_2D = sorted(knn, key=lambda x: x[0])

for i in range(3):
  print(sorted_array_2D[i][1])
# print(knn)
toc = time.time()

# knn_aspe = toc -tic
# print(knn_aspe)

[[144102.69279631227, 0], [143517.4079272449, 1], [145438.89490067307, 2]]
1
0
2
[[144102.69279631227, 0], [143517.4079272449, 1], [145438.89490067307, 2]]
time: 3.74 ms (started: 2023-12-07 14:22:59 +00:00)


**MATCHES WITH THE NEAREST NEIGHBOUR BY EUCLIDEAN METHOD**

In [ ]:
from scipy.spatial import distance
min_dis = []
for i in range(len(D)):
  min_dis.append(distance.euclidean(D[i], query))

# print(min_dis)

print(np.argmin(min_dis))
print(np.argmin(knn))

1
1
time: 4.59 ms (started: 2023-12-07 14:23:18 +00:00)


In [ ]:
print(total_encription_time)
# q_doubledash = []

# for i in range(len(query)):
#   q_doubledash.append(beta2*query[i])

# q_doubledash.append(1)

# for i in range(len(x)):
#   q_doubledash.append(x[i])

# for i in range(epsilon):
#   q_doubledash.append(0)

# q_doubledash = np.transpose(q_doubledash)

# err = []
# dot_product = np.dot(E_matrix, N_dash_inv)

# sum=0

# for i in range(len(dot_product)):
#   for j in range(len(dot_product[0])):
#     sum = sum + dot_product[i][j]
#   err.append(sum)
# # print(err)

# for i in range(len(p_tilda_i)-1):

#   erri = np.dot(pi_doubledash[i], err)
#   erri1 = np.dot(pi_doubledash[i+1], err)

#   if ( np.dot((p_tilda_i[i] - p_tilda_i[i+1]), q_doubledash) > (erri - erri1) ):
#     print("YES")
#   else:
#     print("NO")


In [ ]:

# import time
# from google.colab import drive

# # Mount Google Drive
# drive.mount('/content/drive')


# # Record the start time
# start_time = time.time()

# # Run your notebook using the %run magic command
# %run "/content/drive/My Drive/Colab Notebooks/Paillier.ipynb"

# # Record the end time
# end_time = time.time()

# # Calculate the elapsed time
# elapsed_time = end_time - start_time

# # Print the runtime
# print(f"Runtime: {elapsed_time} seconds")
